## Recommendation Engines :



In [38]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules,apriori

### let's read our datasets 

In [39]:
sales_reciepts = pd.read_csv('datasets/201904 sales reciepts.csv')
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [40]:
product = pd.read_csv('datasets\product.csv')
product.head()

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\user\AppData\Local\Temp\ipykernel_17432\1782373845.py:1: SyntaxWarning: invalid escape sequence '\p'
  product = pd.read_csv('datasets\product.csv')


,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


### Let's choose useful columns and merge our 2 tables :

In [41]:
sales_reciepts = sales_reciepts[['transaction_id','transaction_date','sales_outlet_id','customer_id','product_id','quantity']]
product = product[['product_id','product_category','product']]

dataset = pd.merge(sales_reciepts , product ,on ='product_id',how ='left')
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


### Some cleaning
#### let's start with product column

In [42]:
dataset['product'].nunique()

80

In [43]:
dataset['product'].unique()

array(['Traditional Blend Chai Rg', 'Brazilian Lg',
       'Serenity Green Tea Rg', 'Our Old Time Diner Blend Rg',
       'Jamaican Coffee River Sm', 'Ethiopia Rg', 'English Breakfast Lg',
       'Sustainably Grown Organic Rg', 'Earl Grey Lg',
       'Jamaican Coffee River Rg', 'Serenity Green Tea Lg',
       'Brazilian Sm', 'English Breakfast Rg',
       'Traditional Blend Chai Lg', 'Cappuccino', 'Espresso shot',
       'Cappuccino Lg', 'Latte', 'Earl Grey Rg', 'Dark chocolate Lg',
       'Columbian Medium Roast Sm', 'Oatmeal Scone',
       'Morning Sunrise Chai Lg', 'Morning Sunrise Chai Rg',
       'Peppermint Lg', 'Jumbo Savory Scone', 'Lemon Grass Lg',
       'Sustainably Grown Organic Lg', 'Dark chocolate Rg',
       'Lemon Grass Rg', 'Ethiopia Sm', 'Latte Rg',
       'Our Old Time Diner Blend Sm', 'Chocolate Chip Biscotti',
       'Columbian Medium Roast Rg', 'Ethiopia Lg', 'Brazilian Rg',
       'Columbian Medium Roast Lg', 'Spicy Eye Opener Chai Rg',
       'Ginger Biscotti', 

In [44]:
dataset['product'] = dataset['product'].str.replace("Rg"," ")
dataset['product'] = dataset['product'].str.replace("Lg"," ")
dataset['product'] = dataset['product'].str.replace("Sm"," ")

In [45]:
dataset['product'].nunique()

60

### Exclude unnecessary and weird values:

In [46]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [47]:
dataset = dataset[dataset['product'].isin(products_to_take)]

In [48]:
dataset['product'].nunique()

18

 > - oh! Very Good

### Transaction column

In [49]:
dataset['transaction'] = dataset["transaction_id"].astype(str) + "_" + dataset["customer_id"].astype(str)
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131
23,137,2019-04-01,3,264,40,1,Coffee,Cappuccino,137_264


In [50]:
dataset['transaction'].value_counts()

transaction
209_0       31
206_0       29
204_0       27
208_0       25
203_0       24
            ..
19_5572      1
13_5841      1
448_184      1
446_293      1
763_8030     1
Name: count, Length: 6753, dtype: int64

 > - #### Let's exclude customers who bought 1 items ,since it's unuseful for our recommendation engines.

In [51]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
num_of_items_for_each_transaction 

,transaction,count
0,209_0,31
1,206_0,29
2,204_0,27
3,208_0,25
4,203_0,24
...,...,...
6748,19_5572,1
6749,13_5841,1
6750,448_184,1
6751,446_293,1


In [52]:
num_of_items_for_each_transaction[num_of_items_for_each_transaction['count']==1]

,transaction,count
1962,495_383,1
1963,1367_8182,1
1964,1192_0,1
1965,275_105,1
1966,1257_8341,1
...,...,...
6748,19_5572,1
6749,13_5841,1
6750,448_184,1
6751,446_293,1


In [53]:
valid_transaction = num_of_items_for_each_transaction[(num_of_items_for_each_transaction['count']>1)]['transaction'].tolist()

In [54]:
dataset = dataset[dataset['transaction'].isin(valid_transaction)]
dataset.shape

(7346, 9)

### First recommendation engine : Popularity Recommendation Engine
>- This is useful for customers who visit our shop for the first time since we don't have any information about them.

In [55]:
dataset['transaction_id']

251      1185
252      1185
258      1207
259      1207
295      1355
         ... 
49856     680
49880     736
49884     742
49885     746
49886     746
Name: transaction_id, Length: 7346, dtype: int64

In [56]:
popularity_recommendation = dataset.groupby(['product','product_category']).count().reset_index()
popularity_recommendation

,product,product_category,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,transaction
0,Almond Croissant,Bakery,316,316,316,316,316,316,316
1,Cappuccino,Coffee,645,645,645,645,645,645,645
2,Carmel syrup,Flavours,486,486,486,486,486,486,486
3,Chocolate Chip Biscotti,Bakery,315,315,315,315,315,315,315
4,Chocolate Croissant,Bakery,601,601,601,601,601,601,601
5,Chocolate syrup,Flavours,506,506,506,506,506,506,506
6,Cranberry Scone,Bakery,308,308,308,308,308,308,308
7,Croissant,Bakery,330,330,330,330,330,330,330
8,Dark chocolate,Packaged Chocolate,19,19,19,19,19,19,19
9,Espresso shot,Coffee,620,620,620,620,620,620,620


In [57]:
popularity_recommendation= popularity_recommendation[['product','product_category','transaction_id']]
popularity_recommendation = popularity_recommendation.rename(columns = {"transaction_id":"number_of_transactions"})
popularity_recommendation.head()

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,316
1,Cappuccino,Coffee,645
2,Carmel syrup,Flavours,486
3,Chocolate Chip Biscotti,Bakery,315
4,Chocolate Croissant,Bakery,601


In [58]:
popularity_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv',index=False)

### Second recommendation engine : Apriori Recommendation Engine
>- The Apriori Algorithm is usful for discovering frequent itemsets and association rules in customers transactional data.

In [59]:
train_basket = (dataset.groupby(["transaction","product"]))['product'].count().reset_index(name='Count')
train_basket.head()

,transaction,product,Count
0,1001_8306,Cappuccino,1
1,1001_8306,Carmel syrup,1
2,1002_0,Carmel syrup,1
3,1002_0,Espresso shot,1
4,1002_0,Ginger Scone,1


In [60]:
my_basket = train_basket.pivot_table(index='transaction',columns='product',values='Count').fillna(0)
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
1005_5559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [61]:
def encode_units(x):
    if x <= 0:
       return 0
    if x > 0:
        return 1
    
my_basket_sets = my_basket.applymap(encode_units)
my_basket_sets.head(10)

C:\Users\user\AppData\Local\Temp\ipykernel_17432\753693600.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  my_basket_sets = my_basket.applymap(encode_units)


product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1005_5559,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1006_0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1007_8375,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1008_0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1009_5183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


>- #### this is the format which apriori algorthim excpected

In [62]:
frequent_items = apriori(my_basket_sets , min_support=0.05 ,use_colnames =True)
frequent_items.head(15)

c:\Users\user\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.140163,(Almond Croissant)
1,0.282365,(Cappuccino)
2,0.219674,(Carmel syrup)
3,0.133028,(Chocolate Chip Biscotti)
4,0.165138,(Chocolate Croissant)
5,0.223242,(Chocolate syrup)
6,0.135066,(Cranberry Scone)
7,0.141182,(Croissant)
8,0.278287,(Espresso shot)
9,0.130479,(Ginger Biscotti)


In [63]:
rules_basket = association_rules(frequent_items ,metric ='lift',min_threshold=1)
rules_basket.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Carmel syrup),(Cappuccino),0.219674,0.282365,0.076453,0.348028,1.232546,0.014424,1.100714,0.241785
1,(Cappuccino),(Carmel syrup),0.282365,0.219674,0.076453,0.270758,1.232546,0.014424,1.070051,0.262907
2,(Chocolate syrup),(Cappuccino),0.223242,0.282365,0.086646,0.388128,1.374561,0.023611,1.172851,0.350811
3,(Cappuccino),(Chocolate syrup),0.282365,0.223242,0.086646,0.306859,1.374561,0.023611,1.120636,0.379713
4,(Hazelnut syrup),(Cappuccino),0.199796,0.282365,0.071356,0.357143,1.264827,0.014940,1.116321,0.261656
5,(Cappuccino),(Hazelnut syrup),0.282365,0.199796,0.071356,0.252708,1.264827,0.014940,1.070804,0.291761
6,(Sugar Free Vanilla syrup),(Cappuccino),0.231906,0.282365,0.092253,0.397802,1.408823,0.026771,1.191693,0.377802
7,(Cappuccino),(Sugar Free Vanilla syrup),0.282365,0.231906,0.092253,0.326715,1.408823,0.026771,1.140815,0.404367
8,(Carmel syrup),(Espresso shot),0.219674,0.278287,0.081549,0.371230,1.333979,0.020417,1.147816,0.320844
9,(Espresso shot),(Carmel syrup),0.278287,0.219674,0.081549,0.293040,1.333979,0.020417,1.103777,0.346901


In [64]:
rules_basket[rules_basket['antecedents']=={'Latte'}].sort_values("confidence",ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
27,(Latte),(Sugar Free Vanilla syrup),0.281346,0.231906,0.085627,0.304348,1.312375,0.020381,1.104135,0.331206
11,(Latte),(Carmel syrup),0.281346,0.219674,0.082569,0.293478,1.335973,0.020765,1.104462,0.349934
17,(Latte),(Chocolate syrup),0.281346,0.223242,0.074924,0.266304,1.192898,0.012116,1.058693,0.225011
24,(Latte),(Hazelnut syrup),0.281346,0.199796,0.073904,0.262681,1.314746,0.017692,1.085289,0.333118
13,(Latte),(Chocolate Croissant),0.281346,0.165138,0.051988,0.184783,1.118961,0.005527,1.024098,0.147935
19,(Latte),(Croissant),0.281346,0.141182,0.050459,0.179348,1.270326,0.010738,1.046506,0.296110


### Save in JSON format 

In [65]:
product_categories = dataset[['product','product_category']].drop_duplicates().set_index('product').to_dict()['product_category']
product_categories

{'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Cappuccino': 'Coffee',
 'Chocolate Croissant': 'Bakery',
 'Latte': 'Coffee',
 'Croissant': 'Bakery',
 'Cranberry Scone': 'Bakery',
 'Chocolate Chip Biscotti': 'Bakery',
 'Jumbo Savory Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours',
 'Dark chocolate': 'Packaged Chocolate'}

In [66]:
recommendation_json = {}               

antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents']== antecedent]
    df_rec = rules_basket.sort_values("confidence",ascending=False)

    key = '_'.join(antecedent)

    recommendation_json[key] = []
    for _,row in df_rec.iterrows():
        rec_objects = row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendation_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists = True
            if already_exists:
                continue
            rec = {
                'product': rec_object ,
                'product_categories' : product_categories[rec_object],
                'confidence' : row['confidence']
            }
            recommendation_json[key].append(rec)


In [67]:
import pprint
pprint.pp(recommendation_json)

{'Carmel syrup': [{'product': 'Cappuccino',
                   'product_categories': 'Coffee',
                   'confidence': 0.3978021978021978},
                  {'product': 'Latte',
                   'product_categories': 'Coffee',
                   'confidence': 0.37587006960556846},
                  {'product': 'Espresso shot',
                   'product_categories': 'Coffee',
                   'confidence': 0.37244897959183676},
                  {'product': 'Sugar Free Vanilla syrup',
                   'product_categories': 'Flavours',
                   'confidence': 0.32671480144404336},
                  {'product': 'Chocolate syrup',
                   'product_categories': 'Flavours',
                   'confidence': 0.3068592057761733},
                  {'product': 'Carmel syrup',
                   'product_categories': 'Flavours',
                   'confidence': 0.29347826086956524},
                  {'product': 'Hazelnut syrup',
                   'product_c

In [70]:
import json 
with open('api/recommendation_objects/apriori_recommendation.json','w') as json_file:
    json.dump(recommendation_json , json_file)

### Done!